# Нейронные сети
__Суммарное количество баллов: 10__

В этом домашнем задании можно использовать готовые классы библиотек PyTorch, Keras, TensorFlow.

In [1]:
import keras


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

In [91]:
def LoadMnist():
    
    num_classes = 10

    from keras.datasets import mnist
    # input image dimensions
    img_rows, img_cols = 28, 28

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()


    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    return (x_train, y_train), (x_test, y_test)

### Задание 1 (4 балла)

Создайте сеть из трех сверточных слоев по 8 сверток 3х3 и двух полносвязных слоев по 64 нейрона. 

Обучите сеть на датасете mnist с тремя разными функциями активации в слоях (sigmoid, tanh, ReLU).

In [90]:
def FitMnist(train_data, validation_data, activation = 'relu'):
    
    model = keras.Sequential([
    keras.layers.Conv2D(8, kernel_size = (3,3), input_shape=(28, 28, 1), activation=activation),
    keras.layers.Conv2D(8, kernel_size = (3,3), activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation=activation),
    keras.layers.Dense(64, activation=activation),
    keras.layers.Dense(num_classes, activation="softmax")])

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    
    x_t, y_t = train_data
    model.fit(x_t, y_t, validation_data = validation_data)
    
    return model


In [132]:
train_data, test_data = LoadMnist()
FitMnist(train_data, test_data, 'relu')

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 23s 391us/step - loss: 0.1598 - accuracy: 0.9500 - val_loss: 0.0659 - val_accuracy: 0.9796


In [134]:
FitMnist(train_data, test_data, 'tanh')

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 28s 470us/step - loss: 0.2047 - accuracy: 0.9399 - val_loss: 0.0999 - val_accuracy: 0.9713


In [136]:
FitMnist(train_data, test_data, 'sigmoid')

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 26s 439us/step - loss: 2.3082 - accuracy: 0.1045 - val_loss: 2.3075 - val_accuracy: 0.1032


### Задание 2 (6 баллов)

Достигните точности 87% на test датасете notMNIST.

Архитектура сети может быть любая. Можно использовать Data Augmentation.

In [80]:
import os
from PIL import Image
import numpy as np

def read_notMNIST(images_dir='notMNIST_small'):
    X = []
    labels = []
    DATA_PATH = images_dir
    # for each folder (holding a different set of letters)
    for directory in os.listdir(DATA_PATH):
        # for each image
        for image in os.listdir(DATA_PATH + '/' + directory):
            # open image and load array data
            try:
                file_path = DATA_PATH + '/' + directory + '/' + image
                img = Image.open(file_path)
                img.load()
                img_data = np.asarray(img, dtype=np.int16)
                # add image to dataset
                X.append(img_data)
                # add label to labels
                labels.append(directory)
            except:
                None # do nothing if couldn't load file
    N = len(X)
    X = np.asarray(X).reshape(N, 28, 28, 1) # add our single channel for processing purposes
    labels = keras.utils.to_categorical(list(map(lambda x: ord(x)-ord('A'), labels)), 10) # convert to one-hot
    return X, labels


In [130]:
from keras.preprocessing.image import ImageDataGenerator

def FitNoMnist(train_data, validation_data, activation = 'relu', batch_size = 32, epochs = 50):
    
    model = keras.Sequential([
    keras.layers.Conv2D(8, kernel_size = (3,3), input_shape=(28, 28, 1), activation=activation),
    keras.layers.Conv2D(8, kernel_size = (3,3), activation=activation),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation=activation),
    keras.layers.Dropout(rate = 0.05),
    keras.layers.Dense(256, activation=activation),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(num_classes, activation="softmax")])

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    
    datagen = ImageDataGenerator(
        rotation_range=15,
        zoom_range = 0.15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=False,
        vertical_flip=False)

    xTrain, yTrain = train_data
    xTest, yTest = validation_data
    
    train_gen = datagen.flow(xTrain, yTrain, batch_size=batch_size)
    test_gen = datagen.flow(xTest, yTest, batch_size=batch_size)
    
    model.fit_generator(train_gen, validation_data = test_gen, epochs = epochs,
                       steps_per_epoch = X_train.shape[0] // batch_size,
                       validation_steps = X_train.shape[0] // batch_size)
    
    return model


In [131]:
from sklearn.model_selection import train_test_split

X, y = read_notMNIST()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.11)

FitNoMnist((X_train, y_train), (X_test, y_test), 'tanh')

Epoch 1/50
520/520 [==============================] - 11s 21ms/step - loss: 0.8349 - accuracy: 0.7448 - val_loss: 0.7635 - val_accuracy: 0.8197
Epoch 2/50
520/520 [==============================] - 10s 20ms/step - loss: 0.6071 - accuracy: 0.8186 - val_loss: 0.5667 - val_accuracy: 0.8202
Epoch 3/50
520/520 [==============================] - 11s 20ms/step - loss: 0.5260 - accuracy: 0.8444 - val_loss: 0.3538 - val_accuracy: 0.8660
Epoch 4/50
520/520 [==============================] - 11s 20ms/step - loss: 0.4665 - accuracy: 0.8601 - val_loss: 0.2418 - val_accuracy: 0.8752
Epoch 5/50
520/520 [==============================] - 11s 21ms/step - loss: 0.4433 - accuracy: 0.8678 - val_loss: 0.4694 - val_accuracy: 0.8754
Epoch 6/50
520/520 [==============================] - 11s 21ms/step - loss: 0.4223 - accuracy: 0.8757 - val_loss: 0.0748 - val_accuracy: 0.8776
Epoch 7/50
520/520 [==============================] - 11s 21ms/step - loss: 0.4056 - accuracy: 0.8796 - val_loss: 0.1506 - val_accuracy: